# Data augmentation

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

Mounted at /content/drive


In [2]:
os.chdir('/content/drive/My Drive/resources_for_graduate')

In [3]:
!pip install -q tf-models-official==2.4.0

     |████████████████████████████████| 1.1MB 5.3MB/s 
     |████████████████████████████████| 645kB 18.2MB/s 
     |████████████████████████████████| 1.2MB 12.5MB/s 
     |████████████████████████████████| 686kB 37.7MB/s 
     |████████████████████████████████| 358kB 39.6MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 38.2MB 1.2MB/s 
     |████████████████████████████████| 102kB 13.5MB/s 
     |████████████████████████████████| 215kB 54.7MB/s 


## import

In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks

# 내가 임의로 추가
import pandas as pd
import tensorboard
from sklearn.model_selection import train_test_split

# triplet
import tensorflow_addons as tfa
import tensorflow.keras.backend as K

## resources (config)

In [5]:
aug_nums = [29,39,49,99,149]

for aug_num in aug_nums:
    # OUTPUT: 데이터가 저장될 경로, 모델명, 버전 정보
  model_name = 'TRI' # 여기도 변경
  version = 'v4.0.0' 
  pick_num = 4 # 4개 고를거다 (original 문장 외 4개 추가로 고를 것임)

  # INPUT: 기존 BERT base 모델, 데이터 경로
  model_ckpt = 'uncased_L-12_H-768_A-12' # ckpt, vocab 들어있는 폴더 (구글 공식 제공)
  dataset = 'trec' # 데이터 폴더

  # 불러올 모델의 이름, epoch, 버전 설정
  output_dir = 'output' # 따로 없고 trec 폴더로 하면 될 듯
  encoder_name = 'TRI_TREC_6'
  encoder_epoch_num = 1 # 최저 val_loss epoch
  encoder_version = 'v2.5'

  # dataset 및 shape 설정
  dataset_type = 'TREC_6' # TREC_6 / TREC_42
  ftmodel_shape = 'L2' # ORI / L2 / DENSE_L2
  loss_type = 'TRI' # TRI / COS_OHOT / COS_OMNS 💥

  # CONFIG 출력

  dataoutput_dir = dataset + '/DA/' + '%s/%s_%d_%s_%d'%(dataset_type, model_name, pick_num, version, aug_num+1)
  my_encoder_ckpt = output_dir + '/' + encoder_name + '/' + encoder_version + '/models/encoder/%s'%encoder_epoch_num

  print('==== LOAD ENCODER 경로 ====')
  print('load encoder path:', my_encoder_ckpt, '\n')

  print('==== OUTPUT 경로 ====')
  print('data output path:', dataoutput_dir)

  # 에러 메시지 안띄움
  tf.get_logger().setLevel('FATAL')

  # df 불러오기
  train_df = pd.read_csv(dataset+'/preDA/%d_eda_trec_train_v4.0.csv'%(aug_num+1))
  test_df = pd.read_csv(dataset+'/trec_test.csv') 

  if dataset_type == 'TREC_6':
      # 6개짜리로 index 수정
      train_df['Class Index'] = train_df['Class Index'].str.split(':').str[0]
      # test_df['Class Index'] = test_df['Class Index'].str.split(':').str[0]

  elif dataset_type == 'TREC_42': # 42면 test도 불러와야 함
      # 클래스 개수 파악
      num_classes = len(train_df['Class Index'].unique())
      print('class개수: %d개'%num_classes)
      print('class목록:',list(train_df['Class Index'].unique()))

      # test에 없는 클래스 파악
      test_list = list(test_df['Class Index'].unique())
      remove_label = []

      for label in list(train_df['Class Index'].unique()):
          if label not in test_list:
              print(label)
              remove_label.append(label)
              
      # test에 없는 클래스 제거
      for remove in remove_label:
          train_df = train_df[train_df['Class Index'] != remove]

      train_df
      
  else:
      raise Exception('잘못된 데이터 셋 입력')

  # 클래스 개수 파악
  num_classes = len(train_df['Class Index'].unique())
  print('class개수: %d개'%num_classes)
  print('class목록:',list(train_df['Class Index'].unique()))

  train_df 

  num = 0
  label_dic = {}
  for i in list(train_df['Class Index'].unique()):
      label_dic[i] = num
      num+=1

  # 필요시 label 수정
  train_df['Class Index'] = train_df['Class Index'].replace(label_dic)
  # test_df['Class Index'] = test_df['Class Index'].replace(label_dic)
  # validation_df['Class Index'] = validation_df['Class Index'].replace(label_dic)

  print('수정된 class목록:',label_dic)

  # label_dic을 거꾸로 하는 dic생성
  reverse_label_dic = {v: k for k, v in label_dic.items()}
  reverse_label_dic

  train_df

  # vocab 불러와서 tokenizer 생성
  # Set up tokenizer to generate Tensorflow dataset
  tokenizer = bert.tokenization.FullTokenizer(
      vocab_file=os.path.join(model_ckpt, "vocab.txt"),
      do_lower_case=True)

  print("Vocab size:", len(tokenizer.vocab))

  # 위의 과정을 합친 것으로, 함수 두 개 선언
  # 쪼개고, 뒤에 [SEP] 붙이고 ids로 변환
  def encode_sentence(s, tokenizer): 
    tokens = list(tokenizer.tokenize(s))
    tokens.append('[SEP]')
    return tokenizer.convert_tokens_to_ids(tokens)

  # 실제 문장들에 위 함수 적용하고, 맨 앞에 CLS 붙여줌
  def bert_encode(df_column, tokenizer): 
  #   num_examples = len(df_column)
    
    sentence1 = tf.ragged.constant([
        encode_sentence(s, tokenizer)
        for s in np.array(df_column)])

    cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*sentence1.shape[0]
    input_word_ids = tf.concat([cls, sentence1], axis=-1)

  # 실제 텍스트인 부분을 1로 바꿔줌 (텍스트 및 패딩 영역 시각화)
    input_mask = tf.ones_like(input_word_ids).to_tensor()

      # cls~sentence1은 0으로, sentence2는 1로 변경 (시각화)
    type_cls = tf.zeros_like(cls)
    type_s1 = tf.zeros_like(sentence1)
    input_type_ids = tf.concat(
        [type_cls, type_s1], axis=-1).to_tensor(1)

    inputs = {
        'input_word_ids': input_word_ids.to_tensor(),
        'input_mask': input_mask,
        'input_type_ids': input_type_ids
        }

    return inputs

  import json

  bert_config_file = os.path.join(model_ckpt, "bert_config.json")
  config_dict = json.loads(tf.io.gfile.GFile(bert_config_file).read())

  bert_config = bert.configs.BertConfig.from_dict(config_dict)

  config_dict

  if ftmodel_shape == 'ORI': # ORIGINAL인 경우
      # 1_ bert_encoder, classifier 둘 다 불러옴
      bert_classifier, bert_encoder = bert.bert_models.classifier_model(
      bert_config, num_labels=num_classes)
      
      # 불러온 모델 시각화는?
      
      # 초기 출력값
      initial_encoder = bert_encoder(bert_encode(['hi my','name hj'],tokenizer))
      initial_classifier = bert_classifier.predict(bert_encode(['hi my','name hj'],tokenizer))
      
      # 가중치 복원 (내가 학습시킨 encoder)
      bert_encoder.load_weights(os.path.join(my_encoder_ckpt, 'ckpt'))
      
      # checkpoint = tf.train.Checkpoint(model=bert_encoder)
      # checkpoint.restore(os.path.join(my_encoder_ckpt, 'ckpt')).expect_partial() # 에러발생할수도  

      
      # 복원 후 출력값
      real_encoder = bert_encoder(bert_encode(['hi my','name hj'],tokenizer))
      real_classifier = bert_classifier.predict(bert_encode(['hi my','name hj'],tokenizer))
      
      # temp encoder 생성 (encoder 학습 시의 구조 복원)
      from tensorflow.keras.layers import Lambda, Dense

      x = bert_encoder.layers[-1].output
      outputs = Lambda(lambda x: tf.math.l2_normalize(x,axis=1))(x)
      bert_encoder_test = tf.keras.Model(bert_encoder.input, outputs)

      test_encoder = bert_encoder_test(bert_encode(['hi my','name hj'],tokenizer))
      del bert_encoder_test # 다 썼으면 버려
      
      # encoder끼리 다르고, classifier끼리 달라야 함
      print('=== Checklist 1 : 하단 encoder 및 classifier 값이 다른지 ===')
      
      if not np.array_equal(initial_encoder[0],real_encoder[0]) and not np.array_equal(initial_encoder[1],real_encoder[1]):
          print('(1) encoder 가중치 복원 성공 ✨')
      else:
          raise Exception('encoder 가중치 복원 실패')
          
      if not np.array_equal(initial_classifier[0],real_classifier[0]) and not np.array_equal(initial_classifier[1],real_classifier[1]):
          print('(2) classifier 연결 성공 ✨ \n')   
      else:
          raise Exception('classifier 연결 실패')
          
      print('=== Checklist 2 : 하단 encoder 값이 pretrain 시의 값과 같은지 (last epoch인 경우만!) ===')
      print(test_encoder)
      
  elif ftmodel_shape == 'L2': # L2 normalize 계층만 추가된 경우 (BERT-Triplet)
      # 임시 bert_encoder, classifier 불러옴
      _, bert_encoder_temp = bert.bert_models.classifier_model(
      bert_config, num_labels=num_classes)
      
      # bert_encoder 생성 (L2 추가)
      from tensorflow.keras.layers import Lambda, Dense
      
      x = bert_encoder_temp.layers[-1].output # 임시 encoder 마지막 계층의 아웃풋을
      outputs = Lambda(lambda x: tf.math.l2_normalize(x,axis=1))(x) # 받아와서 뒤에 L2 붙여줌
      bert_encoder = tf.keras.Model(bert_encoder_temp.input, outputs) # encoder 생성

      # bert_classifier 생성 (encoder 마지막 계층 뒤에 분류 레이어 부착) => 원래 BERT FT 코드 따라함
      output_temp = tf.keras.layers.Dropout(rate=bert_config.hidden_dropout_prob)(outputs) 
      initializer = tf.keras.initializers.TruncatedNormal(stddev=bert_config.initializer_range)
      output_l2 = tf.keras.layers.Dense(num_classes, kernel_initializer=initializer, name='output')(output_temp)
      
      bert_classifier = tf.keras.Model(
          inputs=bert_encoder.inputs,
          outputs=output_l2)

      del _
      del bert_encoder_temp
      
      # 초기 출력값
      initial_encoder = bert_encoder(bert_encode(['hi my','name hj'],tokenizer))
      initial_classifier = bert_classifier.predict(bert_encode(['hi my','name hj'],tokenizer))
      
      # 가중치 복원 (내가 학습시킨 encoder)
      bert_encoder.load_weights(os.path.join(my_encoder_ckpt, 'ckpt'))
      
      # checkpoint = tf.train.Checkpoint(model=bert_encoder)
      # checkpoint.restore(os.path.join(my_encoder_ckpt, 'ckpt')).expect_partial() # 에러발생할수도  

      
      # 복원 후 출력값
      real_encoder = bert_encoder(bert_encode(['hi my','name hj'],tokenizer))
      real_classifier = bert_classifier.predict(bert_encode(['hi my','name hj'],tokenizer))
      
      # encoder끼리 다르고, classifier끼리 달라야 함
      print('=== Checklist 1 : 하단 encoder 및 classifier 값이 다른지 ===')
      
      if not np.array_equal(initial_encoder[0],real_encoder[0]) and not np.array_equal(initial_encoder[1],real_encoder[1]):
          print('(1) encoder 가중치 복원 성공 ✨')
      else:
          raise Exception('encoder 가중치 복원 실패')
          
      if not np.array_equal(initial_classifier[0],real_classifier[0]) and not np.array_equal(initial_classifier[1],real_classifier[1]):
          print('(2) classifier 연결 성공 ✨ \n')   
      else:
          raise Exception('classifier 연결 실패')
          
      print('=== Checklist 2 : 하단 encoder 값이 pretrain 시의 값과 같은지 (last epoch인 경우만!) ===')
      print(real_encoder) # test encoder가 아닌 real_encoder임

  elif ftmodel_shape == 'DENSE_L2':
      # 임시 bert_encoder, classifier 불러옴
      _, bert_encoder_temp = bert.bert_models.classifier_model(
      bert_config, num_labels=num_classes)
      
      # bert_encoder 생성 (DENSE + L2 추가)
      from tensorflow.keras.layers import Lambda, Dense
      from tensorflow.keras.utils import to_categorical
      from tensorflow.python.ops import math_ops
      from tensorflow.python.ops import nn
      
      x = bert_encoder_temp.layers[-1].output # 임시 encoder 마지막 계층의 아웃풋을
      output_temp = Dense(num_classes, activation=None)(x) # 받아와서 Dense 붙여준 후 ❤ (L2 모델과 비교 시 위 import랑 여기만 다름)
      outputs = Lambda(lambda x: tf.math.l2_normalize(x,axis=1))(output_temp) # 받아와서 뒤에 L2 붙여줌
      bert_encoder = tf.keras.Model(bert_encoder_temp.input, outputs) # encoder 생성
      
      # bert_classifier 생성 (encoder 마지막 계층 뒤에 분류 레이어 부착) => 원래 BERT FT 코드 따라함
      output_temp = tf.keras.layers.Dropout(rate=bert_config.hidden_dropout_prob)(outputs) 
      initializer = tf.keras.initializers.TruncatedNormal(stddev=bert_config.initializer_range)
      output_l2 = tf.keras.layers.Dense(num_classes, kernel_initializer=initializer, name='output')(output_temp)
      
      bert_classifier = tf.keras.Model(
          inputs=bert_encoder.inputs,
          outputs=output_l2)

      del _
      del bert_encoder_temp
      
      # 초기 출력값
      initial_encoder = bert_encoder(bert_encode(['hi my','name hj'],tokenizer))
      initial_classifier = bert_classifier.predict(bert_encode(['hi my','name hj'],tokenizer))
      
      # 가중치 복원 (내가 학습시킨 encoder)
      bert_encoder.load_weights(os.path.join(my_encoder_ckpt, 'ckpt'))
      
      # checkpoint = tf.train.Checkpoint(model=bert_encoder)
      # checkpoint.restore(os.path.join(my_encoder_ckpt, 'ckpt')).expect_partial() # 에러발생할수도  

      
      # 복원 후 출력값
      real_encoder = bert_encoder(bert_encode(['hi my','name hj'],tokenizer))
      real_classifier = bert_classifier.predict(bert_encode(['hi my','name hj'],tokenizer))
      
      # encoder끼리 다르고, classifier끼리 달라야 함
      print('=== Checklist 1 : 하단 encoder 및 classifier 값이 다른지 ===')
      
      if not np.array_equal(initial_encoder[0],real_encoder[0]) and not np.array_equal(initial_encoder[1],real_encoder[1]):
          print('(1) encoder 가중치 복원 성공 ✨')
      else:
          raise Exception('encoder 가중치 복원 실패')
          
      if not np.array_equal(initial_classifier[0],real_classifier[0]) and not np.array_equal(initial_classifier[1],real_classifier[1]):
          print('(2) classifier 연결 성공 ✨ \n')   
      else:
          raise Exception('classifier 연결 실패')
          
      print('=== Checklist 2 : 하단 encoder 값이 pretrain 시의 값과 같은지 (last epoch인 경우만!) ===')
      print(real_encoder) # test encoder가 아닌 real_encoder임

  else:
      raise Exception('잘못된 model shape 입력')

  # 클래스별 중심벡터 정의
  if loss_type == 'TRI':
    print('TRIPLET: 사전 저장된 dictionary 불러옴')
    import pickle 
    file=open('trec/triplet_42_center_axis_dic',"rb") 
    dic = pickle.load(file)
    file.close
    center_axis = list(dic.values())
    print(len(center_axis))
    print(center_axis)

      
  elif loss_type == 'COS_OHOT':
    print('COS_OHOT: 라벨 one-hot encoding으로 변환함')

    from tensorflow.keras.utils import to_categorical # 얘도 one-hot인지 뭔지에 따라 조건문 줘야지
    center_axis = tf.convert_to_tensor(to_categorical([axis_num for axis_num in range(num_classes)], num_classes=num_classes))
    print(center_axis)

  elif loss_type == 'COS_OMNS':
    print('COS_OMNS: 라벨 one-minus encoding으로 변환함')

    from tensorflow.keras.utils import to_categorical # 얘도 one-hot인지 뭔지에 따라 조건문 줘야지
    temp_center_axis = to_categorical([axis_num for axis_num in range(num_classes)], num_classes=num_classes)
    temp_center_axis = np.where(temp_center_axis==0, -1, temp_center_axis)
    center_axis = tf.convert_to_tensor(temp_center_axis)
    print(center_axis)

  len(center_axis[0])

  import math

  '''유클리디안 함수 정의'''
  def euclidean_distance(vec1, vec2):
      minus = vec2 - vec1
      square = np.square(minus)  # 제곱
      square_sum = square.sum()  # 제곱의 합
      distance = math.sqrt(square_sum) # 루트
      return distance

  '''분산 계산 함수 정의'''
  def find_var(vec_list):
      average = sum(vec_list) / len(vec_list) # 평균
      dev = np.array(vec_list) - average # 편차 (list)
      square_dev = np.square(dev) # 편차의 제곱 (list)
      var = sum(square_dev) / len(square_dev) # 편차의 제곱의 평균 (분산)
      return var

  train_df

  train_df.reset_index(inplace=True, drop=True)
  train_df # 4904개

  # 비교 후 문장 저장
  extract_df = pd.DataFrame(columns = ['Class Index','Sum'])

  for idx in range(aug_num+1,len(train_df)+1,aug_num+1): # 이렇게 하면 20 40 60 80 .. 등등 20의 배수로 109040까지
    print('🌹 현재 index: %d'%idx)

    # 동일 문장 df 추출
    temp_df = train_df.iloc[idx-(aug_num+1):idx]
    # print(temp_df)

    # 하나의 df에 대하여 original 문장 선택
    ori_str = temp_df['Sum'][idx-1]
    # print(ori_str)

    # 해당 set의 label 및 대응되는 축 가져오기
    ori_label = temp_df['Class Index'][idx-1]
    temp_axis = center_axis[ori_label]

    print('(1) original label:', ori_label)
    print('(2) center axis(상위 10차원만):', temp_axis[:10],'\n')

    # original 문장 먼저 저장 💚
    extract_df = extract_df.append({'Class Index':ori_label,'Sum':ori_str}, ignore_index=True)
    # print(extract_df)
    # print('🧡')

    # 증강문장들을 임베딩! (원래 문장은 제외해야 함 Q. 증강된 놈들 중 중복된 건 어케야 함?)
    temp_embeddings = bert_encoder(bert_encode(temp_df['Sum'][:-1], tokenizer))
    # print(temp_embeddings) # 19, 6 차원

    # 증강문장들과 label간의 거리 비교 (원래 문장은 제외해야 함 Q. 증강된 놈들 중 중복된 건 어케야 함? => 아직은 안없애봄) => df 로 중심축과의 거리 비교함
    temp_sort_df = pd.DataFrame(columns = ['idx','dist'])
    str_idx = idx-(aug_num+1)
    for temp_embed in temp_embeddings: # 한 문장의 임베딩 벡터에 대하여 
      dist = euclidean_distance(temp_embed, temp_axis)
      temp_sort_df = temp_sort_df.append({'idx':str_idx, 'dist':dist}, ignore_index=True)
      # print(dist)
      str_idx += 1

      # 이 dist가 제일 작은 네 개의 문장을 고르면 됨
      # dist가 제일 작은 순으로 정렬할까? (아니면 dist 제일 작은 idx 추출해서 idx를 이용해서 그 문장을 뽑아낼까?)

    # print(temp_sort_df)

    # dist가 가장 작은 네 개의 문장 선택
    temp_sort_df.sort_values(by='dist', inplace=True) # 정렬
    # print(temp_sort_df)
    # print(temp_sort_df.iloc[:pick_num]) 
    # print(temp_df.loc[temp_sort_df.iloc[:pick_num]['idx']])

    # pd에 저장해주기 (최후에 합쳐서 csv로 저장) 💚
    extract_df = pd.concat([extract_df,temp_df.loc[temp_sort_df.iloc[:pick_num]['idx']]], ignore_index=True)
    print(extract_df,'\n')

    # test를 위해 3번만 반복했음
    # if idx == 60:
    #   break

    # 전부 다 한 후에 temp들 삭제해줘야 함! (그 전에는 ori랑 쟤랑 붙여야지)
    del temp_df
    del ori_str
    del ori_label
    del temp_axis
    del temp_embeddings
    del temp_sort_df

    print('='*100)

  extract_df['Class Index'] = extract_df['Class Index'].replace(reverse_label_dic)
  print(extract_df)
  print(dataoutput_dir)

  del bert_encoder

  # extract_df를 저장하면 됨
  extract_df.to_csv(dataoutput_dir+'.csv',index=False)
  # extract_df.to_csv('trec/COS_OHOT_4_v2.6.2.csv',index=False)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
2               0                  did serfdom develop in and russia
3               0              how did develop and then leave russia
4               0   russia did serfdom develop in and then leave how
...           ...                                                ...
26125           0  How can I enforce new rules to a group of youn...
26126           0  how i enforce new rules to a group of youngste...
26127           0  how i enforce rules to a group of youngsters w...
26128           0  how can i enforce new principle to a group of ...
26129           0  how can i enforce new rules to a group of smal...

[26130 rows x 2 columns] 

🌹 현재 index: 784050
(1) original label: 3
(2) center axis(상위 10차원만): tf.Tensor(
[-0.03805298 -0.03742062 -0.0403368   0.03631463  0.03923789 -0.03583113
  0.03899631  0.03619939 -0.0394047  -0.04058566], shape=(10,), dtype=float32) 

      Class Index                                                Sum
0            